In [1]:
#!usr/bin/python
import girder_client
import sys
sys.path.append('/home/jvizcarra/Documents/GutmanLab/OCR2QR/SlideManagement/')
from utils import recurseGetResource, recurseGetItems
from pprint import pprint
import urllib
import imghdr
from PIL import Image
from os import path, remove
import scipy
from skimage import filters
from pytesseract import image_to_string
import re

This code is for adding metadata to all fox slides in the candygram (server?). This is done by first
doing OCR on the thumbnail on a fox slide, running the text result through regex, and finally pushing
that metadata to the candygram API.

In [2]:
#log in to candygram
API_URL = "http://candygram.neurology.emory.edu:8080/api/v1"
gc = girder_client.GirderClient(apiUrl=API_URL)
gc.authenticate(interactive=True) 

Login or email: admin
Password for admin: ········


In [34]:
#get the SlideSet
collectionToOrganize_id = '595d08b392ca9a000d0748df' #fox slides
#recurseGetItems does not work with starting collections
#remedy this by pulling out the folders beforehand and running it with each folder in a loop
#appending the results after each iteration
ADRC_SlideSet = []
foldersInCollection = list(gc.listFolder(collectionToOrganize_id, parentFolderType='collection'))
for fds in foldersInCollection:
    ADRC_SlideSet.extend(recurseGetItems(gc,fds['_id']))

In [ ]:
print len(ADRC_SlideSet)

In [ ]:
#use this snippet of code to clear any metadata tag you with to clear for all slides in the set
# for sld in ADRC_SlideSet:
#     gc.addMetadataToItem(sld['_id'], {'bad_ocr_output': None})


In [ ]:
def cleanOCROutput(text):
    text = re.sub('(\s+|\r|\n|\t)',' ',text)
    text = re.sub('([^0-9a-zA-Z\s\.\:])+','', text)
    return text

In [ ]:
print len(failedRegex)
print len(successRegex)

In [ ]:
print 'Slides with no thumbnail image: ',len(noImage)
print 'Slides with OCR completely failing: ',len(blankOCR)
print 'Slides with successful OCR: ',len(successOCR)

In [ ]:
pprint(ADRC_SlideSet)

In [ ]:
text = 'dkfad/d342j_fk  j..f$//$kda adfk'
text = re.sub('\s{2,}',' ',text)
text = re.sub('','',text)
text = re.sub('([^0-9a-zA-Z\s\.])+','', text)
# text = re.sub('/\s+/g',' ',text)
# text = re.sub('/^(\s*)([\W\w]*)(\b\s*$)/g', '$2', text)
# text.s
print text

In [ ]:
text = 'jkdfd dkfjdka   dfjkad dfkjj   jfd'
text = re.sub('(\s+)',' ',text)
print text

In [ ]:
from matplotlib.pyplot import imshow
import numpy as np
%matplotlib inline


In [ ]:
imageURL = 'http://candygram.neurology.emory.edu:8080/api/v1/item/596e286892ca9a000d23e981/tiles/images/label'
urllib.urlretrieve(imageURL, tempFile)

img = Image.open(tempFile, 'r')
imshow(np.asarray(img))

In [36]:
def runOCR(pathURL,thresh,cropParams2):
    urllib.urlretrieve(pathURL, 'temp.jpg')
#     print 'iterator',iterator
#     cropParams2[1] = cropParams2[1] + 5 * iterator

    if imghdr.what('temp.jpg') != None: 
        # Separate RGB arrays
        img = Image.open(file('temp.jpg', 'rb'))
#         print 'check: ', cropParams2
        img = img.crop(cropParams2)
        R, G, B = img.convert('RGB').split()
        r = R.load()
        g = G.load()
        b = B.load()
        w, h = img.size

        # Convert non-black pixels to white
        for i in range(w):
            for j in range(h):
                if(r[i, j] > thresh or g[i, j] > thresh or b[i, j] > thresh):
                    r[i, j] = 255 # Just change R channel

        # Merge just the R channel as all channels
        img = Image.merge('RGB', (R, R, R))
        img.save('temp.jpg')
        img = Image.open('temp.jpg')
#         remove('temp.jpg') #remove the file
        text = image_to_string(img) #perform ocr on the tailored image
        #do regex to remove any unwanted spaces or strange characters
        #sld['meta']['ocr_output'] = re.sub('\s{2,}','',sld['meta']['ocr_output'])
        text = re.sub('([^0-9a-zA-Z\:\s\/])+','', text)
        text = re.sub('\.',' ', text)
        text = re.sub('(\s{1,}|\n)',' ',text)
        text =re.sub(r'.(?i)*Fox', 'Fox', text)
        if  text[0:3] != 'Fox':
            text = re.sub(r'.(?i)*Col', 'Col', text)
        return text
    else:
        return 'No large image for this url'

In [50]:
def cleanFoxMetaDataOutput(dirtyOutput):
    if dirtyOutput['BrainID'] != None:
        dirtyOutput['BrainID'] = re.sub('\s','',dirtyOutput['BrainID']).upper() #remove any spaces from brain ID
        dirtyOutput['BrainID'] = re.sub('FOX','Fox',dirtyOutput['BrainID'])
    
    if dirtyOutput['Hemi'] != None:
        dirtyOutput['Hemi'] = re.sub('(?i)LH','LH',dirtyOutput['Hemi'])
        dirtyOutput['Hemi'] = re.sub('(?i)Cau','Caudal',dirtyOutput['Hemi'])
        dirtyOutput['Hemi'] = re.sub('(?i)LH M','LH Mid',dirtyOutput['Hemi'])
        dirtyOutput['Hemi'] = re.sub('(?i)(LH F|LHF)','LH Frontal',dirtyOutput['Hemi'])
        
    dirtyOutput['Stain'] = re.sub('(?i)(AVP)','AVP 1:20k',dirtyOutput['Stain'])
    dirtyOutput['Stain'] = re.sub('(?i)(ss|lesl)','Nissl',dirtyOutput['Stain'])
    dirtyOutput['Stain'] = re.sub('(?i)(Slver|Silver)','Silver',dirtyOutput['Stain'])
    
    dirtyOutput['StainDate'] = re.sub('\/{1,}','/',dirtyOutput['StainDate'])
    dirtyOutput['StainDate'] = re.sub('\s','',dirtyOutput['StainDate'])
    return dirtyOutput

In [ ]:
# startImageURL = 'http://candygram.neurology.emory.edu:8080/api/v1/item/'
# endImageURL = '/tiles/images/label?height=500'
# sample = startImageURL + '596e284492ca9a000d23e8b1' + endImageURL
# # print runOCR(sample, 100, [70,25,425,500])
# cropParams = [70,0,425,300]

# print runOCR(sample, 70, cropParams)
# # Fox17u LH Mid 1 Cd. .7 S. .7 Nissl 4/12/17

In [ ]:
#testing: save the image of the first slide's thumbnail, check
#testing: run ocr on the image and spit out the output, check
#testing: do after OCR cleanup on the output (this is some regex), check
    #note that I still have some weird white space character I can't take care of
#add the non-regex ocr-output API, (takes a while...don't run the addMetaDataToItem line until the end), check
#testing: check how many OCRs came out blank, check
#testing: run the specialized regex to pull out the blocks desired
    #pass it through Gutman's regex from meeting with Olivia & Erin
startImageURL = 'http://candygram.neurology.emory.edu:8080/api/v1/item/'
endImageURL = '/tiles/images/label?height=500'
thresh = 95 #the threshold for the black and white conversion
cropParams = [70,0,425,300]

#to play with later without having to run everything
successOCR = []
blankOCR = []
noImage = []
failedRegex = []
successRegex = []

# sampleRegex = '(?P<BrainID>Fox ?\d{1,2} ?.)? ?(?P<Hemi>(LW|LH Mid|LH F|LH|Caud|Cau))?.*(Col.|Cd.|Co.|Cd|Co|Col) ?(?P<Column>\d) (?i)(3.|8.|s.|3|8|s) ?(?P<Section>\d+\w?).*(?P<Stain>(?i)(ss|Slver|Silver|lesl|AVP 120k|AVP 1:20k|AVP)).* (?P<StainDate>(\d{1,2}\/\d{1,2}\/\d{1,2}|\d{1,2}\/\/?\d{1,2}))'
# sampleRegex = '(?P<BrainID>Fox ?\d ?\d?( )?(?i)(u|t|a))? ?(?P<Hemi>(?i)(LH M|LH F|LH Cau|LHF|Cau))?.* ?(?i)(Cd|Co|Col) ?(?P<Column>(0|1|2|3|4|5|6|7|9)) (?i)(3|8|s) ?(?P<Section>\d{1,3}b?).*(?P<Stain>(?i)(ss|Slver|Silver|lesl|AVP)).* (?P<StainDate>(\d ?\d?\/\d ?\d?\/\1 ?(6|7)|\d ?\d?\/1 ?(6|7)))'
sampleRegex = '(?P<BrainID>Fox ?\d ?\d?( )?(?i)(u|t|a))? ?(?P<Hemi>(?i)(LH M|LH F|LH Cau|LHF|Cau))?.* ?(?i)(Cd|Co|Col) ?(?P<Column>(0|1|2|3|4|5|6|7|9)) (?i)(3|8|s) ?(?P<Section>\d{1,3}b?).*(?P<Stain>(?i)(ss|Slver|Silver|lesl|AVP)).* (?P<StainDate>(\d\d?\/\d\d?\/1(6|7)|\d\d?\/1(6|7)))'

re_ADRC_SlideSet = re.compile(sampleRegex)

#set up the meta data to add to slides with no valid image
# and those that have no regex match
noValidImage = {'ocr_output':'No valid Image', 'groups': None}
noRegexMatch = {'ocr_output':'Try ocr again', 'groups':{'BrainID':None,'Hemi':None,'Column':None,'Section':None,'Stain':None,'StainDate':None}}

# ADRC_SlideSetCurrent = ADRC_SlideSet[35:]
juanscounter = 0
i = juanscounter
for sld in ADRC_SlideSet[juanscounter:]:
    tempOutput = []
    i = i + 1
    cropParams[1] = 0
    j= 0
    imageURL = startImageURL + sld['_id'] + endImageURL
    output = runOCR(imageURL,thresh,cropParams) 
    m = re_ADRC_SlideSet.search(output)
    
    if output != 'No large image for this url':
    
        while j < 5 and m == None:
            j = j + 1
            cropParams[1] = j*5
            output = runOCR(imageURL,thresh,cropParams)
            tempOutput.append(output)
            m = re_ADRC_SlideSet.search(output)
     
        cropParams[1] = 0
        j= 0
        while j <= 5 and m == None:
            cropParams[1] = j*5
            j = j + 1
            output = runOCR(imageURL,thresh-20,cropParams)
            tempOutput.append(output)
            m = re_ADRC_SlideSet.search(output)

        if m == None:
            if len(output) == 0:
                blankOCR.append(sld)
            else:
                failedRegex.append(sld)
            print 'FAILED'
            print 'Slide ',i,' ', sld['_id']
            pprint(tempOutput) 
            print '\n'
            gc.addMetadataToItem(sld['_id'], noRegexMatch)
        else:
#             print 'Slide ',i,' was a success'
#             output =cleanFoxMetaDataOutput(output)
            sld['meta']['ocr_output'] = output
            sld['meta']['groups'] = m.groupdict()
            sld['meta']['groups'] = cleanFoxMetaDataOutput(sld['meta']['groups'])
            successRegex.append(sld)
            gc.addMetadataToItem(sld['_id'],sld['meta'])
    else:
        noImage.append(sld)
        gc.addMetadataToItem(sld['_id'], noValidImage)
#     break
        
#         print 'Slide ',i, ' ', sld['_id'],output
#         print m.groupdict()
#     gc.addMetadataToItem(sld['_id'], {'ocr_output':output})
#     if sld['meta'].has_key('ocr_output'):    
#         if len(sld['meta']['ocr_output']) < 1:
#             blankOCR.append(sld['_id'])
#         else:
#             successOCR.append(sld['_id'])
#             print sld['name'],': ',sld['meta']['ocr_output']
            
#             m = re_ADRC_SlideSet.search(sld['meta']['ocr_output'])
#             if m:
# #                 print 'success'
#                 m = m.groupdict()
#                 successRegex.append(m)
#             else:
#                 print sld['name']
#                 print sld['meta']['ocr_output']
#                 failedRegex.append(sld['meta']['ocr_output'])
#     else: 
#         noImage.append(sld['_id'])
    

FAILED
Slide  37   596e285092ca9a000d23e8fb
[u'Col 7 8 451 Nissl 4/1217',
 u'Col 7 8 451 Nissl 4/1217',
 u'Fox17U LH Mid Col 7 Si 451 Nissl 4/12r17',
 u'Fox17U LH Mid Col 7 8 451 Nissl 4/1217',
 u'Fox17U LH Mid Col 7 8 451 Nissl 4/1217',
 u'Fox17U LH Mid Col 7 S 451 Nissl 4/1217',
 u'Col 7 s 451 Nissl 4/1217',
 u'Col 7 s 451 Nissl 4/1217',
 u'Fox17U LH Mid Col 7 S 451 Nissl 4/1217',
 u'Fox17U LH Mid Col 7 S 451 Nissl 4/1217',
 u'Fox17U H Mid Col 7 S 451 Nissl 4/1217']


FAILED
Slide  97   596e286692ca9a000d23e975
[u'Fox56A LH Col 73 535 ti Hlscl 304/17 mam:',
 u'Fox56A LHA Col 7 s 535 t4 Hlscl 304/17 mam:',
 u'FoxbbA will Col 7 s 535 t4 Hlscl 304/17 mam:',
 u'Col 73 535 3 Nissl 5 3/94/17 mam:',
 u'Col 78 535 Hlscl 304/17 mam:',
 u'Fox56A LH Vg Col 78 535 1651 3/94/17 5: F16:',
 u'Col 78 535 lhszl 3/94/17 5: Ht1',
 u'Fox56A LH Col 78 535 lhszl 3/94/17 5: Ht1',
 u'FoxbbA LH Col 73 535 1651 304/17 5: F16:',
 u'Col 73 535 1651 304/17 5: F16:',
 u'Col 73 535 1651 304/17 5: F16:']


FAILED
S

FAILED
Slide  363   596e276e92ca9a000d23e305
[u'mg 17 39 24/05/17 Dako 20034 5183201 GFA m',
 u'17439 24/05/17 Dako 20034 5183201GFA m',
 u'E1 E1739 24/05/17 Dako 20034 5183201GFA m',
 u'E1 24/05/17 Dako 20034 5183201 GFA m',
 u'E1 24/05/17 Dako 20034 5183201 GFA m',
 u'E1739 E1 24/05/17 2 Dako 20034 5183201GFA ma',
 u'E1739 E1 24/05/17 2 Dako 20034 5183201GFA ma',
 u'E1739 E1 24/05/17 2 Dako 20034 5183201GFA ma',
 u'E1739 E1 24/05/17 2 Dako 20034 5183201GFA ma',
 u'E1 24/05/17 2 Dako 20034 5133201 :er ma',
 u'E1 24/05/17 Dako 20034 5183201 GFF m']


FAILED
Slide  364   596e277d92ca9a000d23e36f
[u'E17 39 24/05/17 R Dako 20034 5183202 GFAP 1 m I',
 u'E17 39 24/05/17 R Dako 20034 5183202 GFAP 1 m I',
 u'E17 39 24/05/17 R Dako 20034 5183202 GFAP 1 m I',
 u'E17 39 24/05/17 R Dako 20034 5183202 GFAP 1 m I',
 u'E17 39 24/05/17 R Dako 20034 5183202 GFAP 1 m I',
 u'EVE1739 24/05/17 Dako 20034 5 5183202GFAP 3 m l',
 u'EVE1739 24/05/17 Dako 20034 5 5183202GFAP 3 m l',
 u'EVE1739 24/05/17 Dako 20

In [83]:

noRegexMatch = {'ocr_output':'Try ocr again', 'groups':{'BrainID':None,'Hemi':None,'Column':None,'Section':None,'Stain':None,'StainDate':None}}
pprint(noRegexMatch)


{'groups': {'BrainID': None,
            'Column': None,
            'Hemi': None,
            'Section': None,
            'Stain': None,
            'StainDate': None},
 'ocr_output': 'Try ocr again'}


In [38]:
print len(failedRegex)
print len(blankOCR)
print len(successRegex)

110
47
1485


In [39]:
import json
with open('successFoxSlidesDirty3.json', 'w') as outfile:
    json.dump(successRegex, outfile)

In [ ]:
clear successFoxSlidesClean.json

In [52]:
successRegexClean = []
for sld in successRegex:
    sld['meta']['groups'] = cleanFoxMetaDataOutput(sld['meta']['groups'])
    successRegexClean.append(sld)

import json
with open('successFoxSlidesClean.json', 'w') as outfile:
    json.dump(successRegexClean, outfile)

In [53]:
pprint(successRegexClean)

[{u'_id': u'596e284492ca9a000d23e8b1',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-18T15:24:52.431000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'596e284492ca9a000d23e8ae',
  u'largeImage': {u'fileId': u'596e284492ca9a000d23e8b2',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': u'Fox17U',
                        u'Column': u'7',
                        u'Hemi': u'LH Mid',
                        u'Section': u'7',
                        u'Stain': u'Nissl',
                        u'StainDate': u'4/12/17'},
            u'ocr_output': u'Fox17u LH Mid 1 Cd 7 S 7 Nissl 4/12/17'},
  u'name': u'1008028.svs',
  u'size': 3021892971,
  u'updated': u'2017-08-03T00:59:52.005000+00:00'},
 {u'_id': u'596e284492ca9a000d23e8b3',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection

  u'created': u'2017-07-18T15:25:12.982000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'596e285892ca9a000d23e92a',
  u'largeImage': {u'fileId': u'596e285892ca9a000d23e92c',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': u'Fox56A',
                        u'Column': u'7',
                        u'Hemi': u'LH Mid',
                        u'Section': u'451',
                        u'Stain': u'Nissl',
                        u'StainDate': u'3/24/17'},
            u'ocr_output': u'Fox56A LH Mld Col 7 S 451 NISSI 3/24/17'},
  u'name': u'1008088.svs',
  u'size': 5005238005,
  u'updated': u'2017-08-02T17:59:43.282000+00:00'},
 {u'_id': u'596e285992ca9a000d23e92d',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-18T15:25:13.318000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'596e285

  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'596e285892ca9a000d23e92a',
  u'largeImage': {u'fileId': u'596e287592ca9a000d23e9e2',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': u'Fox17U',
                        u'Column': u'7',
                        u'Hemi': u'LH Mid',
                        u'Section': u'319',
                        u'Stain': u'Nissl',
                        u'StainDate': u'4/12/17'},
            u'ocr_output': u'Fox17U LH Mid Col 7 S 319 Nissl 4/12/17'},
  u'name': u'1008182.svs',
  u'size': 1531755301,
  u'updated': u'2017-08-02T18:01:02.363000+00:00'},
 {u'_id': u'596e287592ca9a000d23e9e3',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-18T15:25:41.593000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'596e285892ca9a000d23e92a',
  u'largeImage': {u'fileId': u'

  u'folderId': u'596e275192ca9a000d23e269',
  u'largeImage': {u'fileId': u'596e276692ca9a000d23e2d5',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': None,
                        u'Column': u'7',
                        u'Hemi': None,
                        u'Section': u'499',
                        u'Stain': u'Nissl',
                        u'StainDate': u'4/12/17'},
            u'ocr_output': u'Col 7 S 499 Nissl 4/12/17 A'},
  u'name': u'1008869.svs',
  u'size': 6094116485,
  u'updated': u'2017-08-02T18:02:04.765000+00:00'},
 {u'_id': u'596e276292ca9a000d23e2c0',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-18T15:21:06.266000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'596e275192ca9a000d23e269',
  u'largeImage': {u'fileId': u'596e276292ca9a000d23e2c1',
                  u'sourceName': u'svs'},
  u'meta': {u'groups'

 {u'_id': u'596e279292ca9a000d23e3d6',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-18T15:21:54.074000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'596e278392ca9a000d23e391',
  u'largeImage': {u'fileId': u'596e279292ca9a000d23e3d7',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': u'Fox56A',
                        u'Column': u'7',
                        u'Hemi': u'LH Mid',
                        u'Section': u'127',
                        u'Stain': u'Nissl',
                        u'StainDate': u'3/24/17'},
            u'ocr_output': u'Fox56A LH Md Col 7 S 127 NISSl 3/24/17'},
  u'name': u'1008999.svs',
  u'size': 3828365371,
  u'updated': u'2017-08-02T18:03:48.254000+00:00'},
 {u'_id': u'596e279892ca9a000d23e3fc',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collecti

...

  u'meta': {u'groups': {u'BrainID': u'Fox15U',
                        u'Column': u'9',
                        u'Hemi': u'LH Frontal',
                        u'Section': u'45',
                        u'Stain': u'AVP 1:20k',
                        u'StainDate': u'5/17'},
            u'ocr_output': u'Fox15U LH F Col 9 s 45 120kAVP 5/17'},
  u'name': u'1010162.svs',
  u'size': 162405985,
  u'updated': u'2017-08-02T18:14:09.944000+00:00'},
 {u'_id': u'596e28b092ca9a000d23eb2d',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-18T15:26:40.592000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'596e287e92ca9a000d23ea00',
  u'largeImage': {u'fileId': u'596e28b092ca9a000d23eb2e',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': None,
                        u'Column': u'9',
                        u'Hemi': None,
                      

  u'updated': u'2017-08-02T18:15:07.782000+00:00'},
 {u'_id': u'596e28d292ca9a000d23ec04',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-18T15:27:14.233000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'596e28b892ca9a000d23eb63',
  u'largeImage': {u'fileId': u'596e28d292ca9a000d23ec05',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': None,
                        u'Column': u'9',
                        u'Hemi': None,
                        u'Section': u'273',
                        u'Stain': u'AVP 1:20k',
                        u'StainDate': u'5/17'},
            u'ocr_output': u'Col 9 S 273 1:20k AVP 5/17 m'},
  u'name': u'1010272.svs',
  u'size': 817881239,
  u'updated': u'2017-08-02T18:15:08.254000+00:00'},
 {u'_id': u'596e28d292ca9a000d23ec06',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',

  u'created': u'2017-07-31T18:03:53.786000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'597f710992ca9a000de617b1',
  u'largeImage': {u'fileId': u'597f710992ca9a000de617b7',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': u'Fox17U',
                        u'Column': u'9',
                        u'Hemi': u'LH Caudal',
                        u'Section': u'465',
                        u'Stain': u'AVP 1:20k',
                        u'StainDate': u'5/17'},
            u'ocr_output': u'Fox17U LH Caud I Col 9 S 465 1:20k AVP 5/17'},
  u'name': u'1010564.svs',
  u'size': 1331912393,
  u'updated': u'2017-08-02T18:16:02.508000+00:00'},
 {u'_id': u'597f710a92ca9a000de617b8',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-31T18:03:54.037000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u

  u'updated': u'2017-08-02T18:17:04.246000+00:00'},
 {u'_id': u'597f714292ca9a000de61970',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-31T18:04:50.568000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'597f710992ca9a000de617b1',
  u'largeImage': {u'fileId': u'597f714292ca9a000de61971',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': None,
                        u'Column': u'9',
                        u'Hemi': None,
                        u'Section': u'225',
                        u'Stain': u'AVP 1:20k',
                        u'StainDate': u'5/17'},
            u'ocr_output': u'Col 9 8225 AVP 1:20k 5/17 dTm'},
  u'name': u'1010658.svs',
  u'size': 773627449,
  u'updated': u'2017-08-02T18:17:04.880000+00:00'},
 {u'_id': u'597f711692ca9a000de61838',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df'

                        u'StainDate': u'5/17'},
            u'ocr_output': u'Col 9 S 321 AVP1120k 5/17 '},
  u'name': u'1010774.svs',
  u'size': 658423161,
  u'updated': u'2017-08-02T18:18:17.121000+00:00'},
 {u'_id': u'597f712592ca9a000de618ae',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-31T18:04:21.933000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'597f710992ca9a000de617b1',
  u'largeImage': {u'fileId': u'597f712592ca9a000de618af',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': u'Fox56A',
                        u'Column': u'9',
                        u'Hemi': u'LH Frontal',
                        u'Section': u'333',
                        u'Stain': u'AVP 1:20k',
                        u'StainDate': u'5/17'},
            u'ocr_output': u'Fox 56A LH F 1 Col 9 S 333 AVP 1:20k 5/17'},
  u'name': u'1010775.svs',
  u

In [ ]:
m = re_ADRC_SlideSet.search('Fox17U LH Mid I Col. 7 S. 19 Nissl 4/12/17')
j = 1
if j < 5 and m == None:
    print True

In [ ]:
print len(ADRC_SlideSet)
print len(ADRC_SlideSetCurrent)

In [ ]:
for nm in successRegex:
    print nm['meta']['ocr_output']

In [51]:
with open('successFoxSlidesDirty3.json') as data_file:    
    successRegex = json.load(data_file)

In [15]:
pprint(data)

[{u'_id': u'596e284492ca9a000d23e8b1',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-18T15:24:52.431000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'596e284492ca9a000d23e8ae',
  u'largeImage': {u'fileId': u'596e284492ca9a000d23e8b2',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': u'Fox17u',
                        u'Column': u'7',
                        u'Hemi': u'LH M',
                        u'Section': u'7',
                        u'Stain': u'ss',
                        u'StainDate': u'4/12/17'},
            u'ocr_output': u'Fox17u LH Mid 1 Cd 7 S 7 Nissl 4/12/17'},
  u'name': u'1008028.svs',
  u'size': 3021892971,
  u'updated': u'2017-08-03T00:59:52.005000+00:00'},
 {u'_id': u'596e284492ca9a000d23e8b3',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  

                        u'Stain': u'ss',
                        u'StainDate': u'3/24/17'},
            u'ocr_output': u'Fox56A LH Mid Col 7 S 151 Nissl 3/24/17'},
  u'name': u'1008113.svs',
  u'size': 4225227017,
  u'updated': u'2017-08-02T18:00:04.325000+00:00'},
 {u'_id': u'596e286192ca9a000d23e95f',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-18T15:25:21.581000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'596e285892ca9a000d23e92a',
  u'largeImage': {u'fileId': u'596e286192ca9a000d23e960',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': u'Fox56A',
                        u'Column': u'7',
                        u'Hemi': u'LH M',
                        u'Section': u'139',
                        u'Stain': u'SS',
                        u'StainDate': u'3/24/17'},
            u'ocr_output': u'Fox56A LH Mid Col 7 S 139 N

  u'description': u'',
  u'folderId': u'596e1d1892ca9a000d6bb75d',
  u'largeImage': {u'fileId': u'596e1d2a92ca9a000d6bb7bd',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': None,
                        u'Column': u'7',
                        u'Hemi': None,
                        u'Section': u'67',
                        u'Stain': u'ss',
                        u'StainDate': u'4/19/17'},
            u'ocr_output': u'Col 7 S 67 Nissl 4/19/17'},
  u'name': u'1008760.svs',
  u'size': 2638120447,
  u'updated': u'2017-08-02T18:01:14.392000+00:00'},
 {u'_id': u'596e1d2a92ca9a000d6bb7be',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-18T14:37:30.689000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'596e1d1892ca9a000d6bb75d',
  u'largeImage': {u'fileId': u'596e1d2a92ca9a000d6bb7bf',
                  u'sourceName': u'svs'},
  u'm

  u'meta': {u'groups': {u'BrainID': None,
                        u'Column': u'7',
                        u'Hemi': None,
                        u'Section': u'475',
                        u'Stain': u'ss',
                        u'StainDate': u'4/19/17'},
            u'ocr_output': u'Col 7 8475 Nissl 4/19/17 m'},
  u'name': u'1008887.svs',
  u'size': 4309652093,
  u'updated': u'2017-08-02T18:02:15.743000+00:00'},
 {u'_id': u'596e276592ca9a000d23e2d0',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-18T15:21:09.301000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'596e275192ca9a000d23e269',
  u'largeImage': {u'fileId': u'596e276592ca9a000d23e2d1',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': u'Fox15U',
                        u'Column': u'7',
                        u'Hemi': u'LH M',
                        u'Section': u'4

  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-18T15:21:41.111000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'596e278392ca9a000d23e391',
  u'largeImage': {u'fileId': u'596e278592ca9a000d23e39b',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': None,
                        u'Column': u'7',
                        u'Hemi': None,
                        u'Section': u'79',
                        u'Stain': u'ss',
                        u'StainDate': u'4/12/17'},
            u'ocr_output': u'IUAIvux vvvW Co 7 S 79 Nissl 4/12/17 '},
  u'name': u'1009033.svs',
  u'size': 612557647,
  u'updated': u'2017-08-02T18:04:11.349000+00:00'},
 {u'_id': u'596e279692ca9a000d23e3f2',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-18T15:21:58.750000+00:00',
  u'creatorId': u'5956b83

...

  u'meta': {u'groups': {u'BrainID': None,
                        u'Column': u'9',
                        u'Hemi': None,
                        u'Section': u'69',
                        u'Stain': u'AVP',
                        u'StainDate': u'5/17'},
            u'ocr_output': u'Col 9 S 69 1:20k AVP 5/17 MIMI'},
  u'name': u'1010256.svs',
  u'size': 208025439,
  u'updated': u'2017-08-02T18:15:00.159000+00:00'},
 {u'_id': u'596e28cd92ca9a000d23ebe6',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-18T15:27:09.815000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'596e28b892ca9a000d23eb63',
  u'largeImage': {u'fileId': u'596e28cd92ca9a000d23ebe7',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': u'Fox4T',
                        u'Column': u'9',
                        u'Hemi': u'LH Cau',
                        u'Section': u'

  u'meta': {u'groups': {u'BrainID': u'Fox4T',
                        u'Column': u'9',
                        u'Hemi': u'LH F',
                        u'Section': u'177',
                        u'Stain': u'AVP',
                        u'StainDate': u'5/17'},
            u'ocr_output': u'Fox4T LH Frontal Col 9 S 177 1:20k AVP 5/17'},
  u'name': u'1010335.svs',
  u'size': 363740395,
  u'updated': u'2017-08-02T18:15:57.052000+00:00'},
 {u'_id': u'596e28e492ca9a000d23ec7e',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-18T15:27:32.359000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'596e28b892ca9a000d23eb63',
  u'largeImage': {u'fileId': u'596e28e492ca9a000d23ec7f',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': u'Fox15U',
                        u'Column': u'9',
                        u'Hemi': u'LH M',
                  

  u'meta': {u'groups': {u'BrainID': u'Fox17U',
                        u'Column': u'9',
                        u'Hemi': u'LH M',
                        u'Section': u'117',
                        u'Stain': u'AVP',
                        u'StainDate': u'5/17'},
            u'ocr_output': u'Fox17U LH Mid Col 9 S 117 AVP 1:20k 5/17 A'},
  u'name': u'1010651.svs',
  u'size': 823455673,
  u'updated': u'2017-08-02T18:17:00.533000+00:00'},
 {u'_id': u'597f713292ca9a000de61900',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-31T18:04:34.173000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'597f710992ca9a000de617b1',
  u'largeImage': {u'fileId': u'597f713292ca9a000de61901',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': None,
                        u'Column': u'9',
                        u'Hemi': None,
                        u'

  u'baseParentType': u'collection',
  u'created': u'2017-07-31T18:04:51.292000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'597f710992ca9a000de617b1',
  u'largeImage': {u'fileId': u'597f714392ca9a000de61977',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': None,
                        u'Column': u'9',
                        u'Hemi': None,
                        u'Section': u'381',
                        u'Stain': u'AVP',
                        u'StainDate': u'5/17'},
            u'ocr_output': u'Col 9 S 381 AVP1120k 5/17 M'},
  u'name': u'1010730.svs',
  u'size': 713851785,
  u'updated': u'2017-08-02T18:17:49.291000+00:00'},
 {u'_id': u'597f712092ca9a000de61884',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-31T18:04:16.357000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u

 {u'_id': u'597f710392ca9a000de61783',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',
  u'created': u'2017-07-31T18:03:47.156000+00:00',
  u'creatorId': u'5956b83b92ca9a000d9c0488',
  u'description': u'',
  u'folderId': u'597f710292ca9a000de61780',
  u'largeImage': {u'fileId': u'597f710392ca9a000de61784',
                  u'sourceName': u'svs'},
  u'meta': {u'groups': {u'BrainID': u'Fox17U',
                        u'Column': u'9',
                        u'Hemi': u'LH Cau',
                        u'Section': u'57',
                        u'Stain': u'AVP',
                        u'StainDate': u'5/17'},
            u'ocr_output': u'Fox17U LH Caud Col 9 S 57 1:20k AVP 5/17'},
  u'name': u'1010816.svs',
  u'size': 148745003,
  u'updated': u'2017-08-02T18:18:41.207000+00:00'},
 {u'_id': u'597f710392ca9a000de61785',
  u'_modelType': u'item',
  u'baseParentId': u'595d08b392ca9a000d0748df',
  u'baseParentType': u'collection',


In [4]:
import json

In [54]:
i = 1
for sld in successRegexClean:
    gc.addMetadataToItem(sld['_id'], sld['meta'])
    print 'Slide ',i,' done!'
    i = i + 1

Slide  1  done!
Slide  2  done!
Slide  3  done!
Slide  4  done!
Slide  5  done!
Slide  6  done!
Slide  7  done!
Slide  8  done!
Slide  9  done!
Slide  10  done!
Slide  11  done!
Slide  12  done!
Slide  13  done!
Slide  14  done!
Slide  15  done!
Slide  16  done!
Slide  17  done!
Slide  18  done!
Slide  19  done!
Slide  20  done!
Slide  21  done!
Slide  22  done!
Slide  23  done!
Slide  24  done!
Slide  25  done!
Slide  26  done!
Slide  27  done!
Slide  28  done!
Slide  29  done!
Slide  30  done!
Slide  31  done!
Slide  32  done!
Slide  33  done!
Slide  34  done!
Slide  35  done!
Slide  36  done!
Slide  37  done!
Slide  38  done!
Slide  39  done!
Slide  40  done!
Slide  41  done!
Slide  42  done!
Slide  43  done!
Slide  44  done!
Slide  45  done!
Slide  46  done!
Slide  47  done!
Slide  48  done!
Slide  49  done!
Slide  50  done!
Slide  51  done!
Slide  52  done!
Slide  53  done!
Slide  54  done!
Slide  55  done!
Slide  56  done!
Slide  57  done!
Slide  58  done!
Slide  59  done!
Slide 

Slide  471  done!
Slide  472  done!
Slide  473  done!
Slide  474  done!
Slide  475  done!
Slide  476  done!
Slide  477  done!
Slide  478  done!
Slide  479  done!
Slide  480  done!
Slide  481  done!
Slide  482  done!
Slide  483  done!
Slide  484  done!
Slide  485  done!
Slide  486  done!
Slide  487  done!
Slide  488  done!
Slide  489  done!
Slide  490  done!
Slide  491  done!
Slide  492  done!
Slide  493  done!
Slide  494  done!
Slide  495  done!
Slide  496  done!
Slide  497  done!
Slide  498  done!
Slide  499  done!
Slide  500  done!
Slide  501  done!
Slide  502  done!
Slide  503  done!
Slide  504  done!
Slide  505  done!
Slide  506  done!
Slide  507  done!
Slide  508  done!
Slide  509  done!
Slide  510  done!
Slide  511  done!
Slide  512  done!
Slide  513  done!
Slide  514  done!
Slide  515  done!
Slide  516  done!
Slide  517  done!
Slide  518  done!
Slide  519  done!
Slide  520  done!
Slide  521  done!
Slide  522  done!
Slide  523  done!
Slide  524  done!
Slide  525  done!
Slide  526

Slide  933  done!
Slide  934  done!
Slide  935  done!
Slide  936  done!
Slide  937  done!
Slide  938  done!
Slide  939  done!
Slide  940  done!
Slide  941  done!
Slide  942  done!
Slide  943  done!
Slide  944  done!
Slide  945  done!
Slide  946  done!
Slide  947  done!
Slide  948  done!
Slide  949  done!
Slide  950  done!
Slide  951  done!
Slide  952  done!
Slide  953  done!
Slide  954  done!
Slide  955  done!
Slide  956  done!
Slide  957  done!
Slide  958  done!
Slide  959  done!
Slide  960  done!
Slide  961  done!
Slide  962  done!
Slide  963  done!
Slide  964  done!
Slide  965  done!
Slide  966  done!
Slide  967  done!
Slide  968  done!
Slide  969  done!
Slide  970  done!
Slide  971  done!
Slide  972  done!
Slide  973  done!
Slide  974  done!
Slide  975  done!
Slide  976  done!
Slide  977  done!
Slide  978  done!
Slide  979  done!
Slide  980  done!
Slide  981  done!
Slide  982  done!
Slide  983  done!
Slide  984  done!
Slide  985  done!
Slide  986  done!
Slide  987  done!
Slide  988

Slide  1377  done!
Slide  1378  done!
Slide  1379  done!
Slide  1380  done!
Slide  1381  done!
Slide  1382  done!
Slide  1383  done!
Slide  1384  done!
Slide  1385  done!
Slide  1386  done!
Slide  1387  done!
Slide  1388  done!
Slide  1389  done!
Slide  1390  done!
Slide  1391  done!
Slide  1392  done!
Slide  1393  done!
Slide  1394  done!
Slide  1395  done!
Slide  1396  done!
Slide  1397  done!
Slide  1398  done!
Slide  1399  done!
Slide  1400  done!
Slide  1401  done!
Slide  1402  done!
Slide  1403  done!
Slide  1404  done!
Slide  1405  done!
Slide  1406  done!
Slide  1407  done!
Slide  1408  done!
Slide  1409  done!
Slide  1410  done!
Slide  1411  done!
Slide  1412  done!
Slide  1413  done!
Slide  1414  done!
Slide  1415  done!
Slide  1416  done!
Slide  1417  done!
Slide  1418  done!
Slide  1419  done!
Slide  1420  done!
Slide  1421  done!
Slide  1422  done!
Slide  1423  done!
Slide  1424  done!
Slide  1425  done!
Slide  1426  done!
Slide  1427  done!
Slide  1428  done!
Slide  1429 